In [64]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import time
import pandas as pd
import gurobipy as grb
import datetime
import random
np.set_printoptions(edgeitems=15,linewidth=600)

%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from src.optimization.MIP.mip_inputs import *
from src.optimization.MIP.mip_model import *
from src.optimization.MIP.mip_outputs import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
UI_params = {
    'initial_procurement_budget':1300000,
    'budget_rate':0.03,
    'initial_operations_budget':1000000,
    'maintenance_budget_rate':0.03,
    'planning_interval':[2022,2037],
    'emissions_baseline': 2031705,#metric tons
    'emissions_target_pct_of_base':0.10,
    'min_miles_replacement_threshold':150000,#miles
    'min_vehicle_age_replacement_threshold':60,#years
    'max_vehicles_per_station':1000,
    'objective_weights':{'cost':0.70,'emissions':0.30},
}

In [67]:
data = pd.read_excel('../data/17MAR_data_template.xlsx').head(25)
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [68]:
inputs = MIP_Inputs(data,UI_params)

In [69]:
inputs.replacement_schedules.shape

(25, 697, 16)

In [6]:
inputs.years

[2022,
 2023,
 2024,
 2025,
 2026,
 2027,
 2028,
 2029,
 2030,
 2031,
 2032,
 2033,
 2034,
 2035,
 2036,
 2037]

In [36]:
example_schedules = [16,10,75]

In [41]:
pd.DataFrame(inputs.replacement_schedules[1,example_schedules,:],columns=inputs.years).astype(int).replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,1,,,,,,,,,,,,,,,
1,,,,,,,1,,,,,,,,,
2,,,,,1,,,,,,,,1,,,


In [42]:
#set of random example schedules
pd.DataFrame(inputs.replacement_schedules[1,example_schedules,:],columns=inputs.years).astype(int)#.replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


In [46]:
pd.DataFrame(inputs.keepSchedules[1,example_schedules,:],columns=inputs.years).astype(int)

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
2,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1


In [44]:
pd.DataFrame(inputs.age[1,example_schedules,:],columns=inputs.years).astype(int)#.replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,20,21,22,23,24,25,0,1,2,3,4,5,6,7,8,9
2,20,21,22,23,0,1,2,3,4,5,6,7,0,1,2,3


In [47]:
pd.DataFrame(inputs.annual_mileage[1,example_schedules,:],columns=inputs.years).astype(int)#.replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641
1,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641
2,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641,3641


In [48]:
pd.DataFrame(inputs.odometer[1,example_schedules,:],columns=inputs.years).astype(int)#.replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,0,3641,7282,10923,14564,18205,21846,25487,29128,32769,36410,40051,43692,47333,50974,54615
1,72820,76461,80102,83743,87384,91025,0,3641,7282,10923,14564,18205,21846,25487,29128,32769
2,72820,76461,80102,83743,0,3641,7282,10923,14564,18205,21846,25487,0,3641,7282,10923


In [49]:
pd.DataFrame(inputs.is_ice[1,example_schedules,:],columns=inputs.years).astype(int)#.replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
pd.DataFrame(inputs.is_ev[1,example_schedules,:],columns=inputs.years).astype(int)#.replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1


In [51]:
pd.DataFrame(inputs.acquisition[1,example_schedules,:],columns=inputs.years).astype(int)#.replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,48500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,48500,0,0,0,0,0,0,0,0,0
2,0,0,0,0,48500,0,0,0,0,0,0,0,48500,0,0,0


In [52]:
pd.DataFrame(inputs.consumables[1,example_schedules,:],columns=inputs.years).astype(int)#.replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130
1,713,713,713,713,713,713,130,130,130,130,130,130,130,130,130,130
2,713,713,713,713,130,130,130,130,130,130,130,130,130,130,130,130


In [53]:
pd.DataFrame(inputs.maintenance[1,example_schedules,:],columns=inputs.years).astype(int)#.replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,1000,1000,1000,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200
1,1200,1200,1200,1200,1200,1200,1000,1000,1000,1200,1200,1200,1200,1200,1200,1200
2,1200,1200,1200,1200,1000,1000,1000,1200,1200,1200,1200,1200,1000,1000,1000,1200


In [55]:
pd.DataFrame(inputs.emissions[1,example_schedules,:],columns=inputs.years)#.astype(int)#.replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.735,0.735,0.735,0.735,0.735,0.735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.735,0.735,0.735,0.735,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
inputs.infeasible_filter[1,example_schedules]+True

array([ True,  True,  True])

In [71]:
m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules = carnet(inputs)

In [72]:
x

{(0, 0): <gurobi.Var C0 (value -0.0)>,
 (0, 1): <gurobi.Var C1 (value -0.0)>,
 (0, 2): <gurobi.Var C2 (value -0.0)>,
 (0, 3): <gurobi.Var C3 (value -0.0)>,
 (0, 4): <gurobi.Var C4 (value -0.0)>,
 (0, 5): <gurobi.Var C5 (value -0.0)>,
 (0, 6): <gurobi.Var C6 (value -0.0)>,
 (0, 7): <gurobi.Var C7 (value -0.0)>,
 (0, 8): <gurobi.Var C8 (value -0.0)>,
 (0, 9): <gurobi.Var C9 (value -0.0)>,
 (0, 10): <gurobi.Var C10 (value -0.0)>,
 (0, 11): <gurobi.Var C11 (value -0.0)>,
 (0, 12): <gurobi.Var C12 (value -0.0)>,
 (0, 13): <gurobi.Var C13 (value -0.0)>,
 (0, 14): <gurobi.Var C14 (value -0.0)>,
 (0, 15): <gurobi.Var C15 (value -0.0)>,
 (0, 16): <gurobi.Var C16 (value -0.0)>,
 (0, 38): <gurobi.Var C17 (value -0.0)>,
 (0, 45): <gurobi.Var C18 (value -0.0)>,
 (0, 46): <gurobi.Var C19 (value -0.0)>,
 (0, 53): <gurobi.Var C20 (value -0.0)>,
 (0, 54): <gurobi.Var C21 (value -0.0)>,
 (0, 55): <gurobi.Var C22 (value -0.0)>,
 (0, 62): <gurobi.Var C23 (value -0.0)>,
 (0, 63): <gurobi.Var C24 (value -0.